# Column
id, title, author, genre, story, platform, age, rate, free, complete, thumbnail, website
    - id: 인덱스
    - title: 제목
    - author: 작가
    - genre: 장르
    - story: 줄거리
    - platform: 플랫폼 (네이버, 다음, 카카오, 레진코믹스 중 1)
    - age: 연령대
    - rate: 평점
    - free: 유료 여부 (0: 유료, 1: 무료)
    - complete: 완결여부
    - thumbnail: 썸네일
    - website: 웹 주소

## 연재 웹툰

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

import pandas as pd

url = 'http://webtoon.daum.net/'
week = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
week_match = '#day=&tab=day'

# 로그인 정보
email = ''
pw = ''

title = []
total_tc = 0
now_tc = 0 

author = {}
genre = {}
story = {}
platform = {}
age = {}
rate = {}
cuttoon = {}
thumbnail = {}
website = {}


for d in week:
    link = url + week_match[0:5] + d + week_match[5:] 
    #print(link)
    
    # webdriver_manager를 사용해 execution_path 변수를 설정
    # 스크립트를 실행할 때 자동으로 드라이버 다운로드
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(link)
    
    # 개발자도구에서 보이는 source code 가져옴
    html = driver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')

    # title, age
    titles = bsObject.find_all('strong', {'class':'tit_wt'}) # find_all('태그명', {'속성명': '값' ...})
    now_tc = len(titles)
    print(f'now_tc={now_tc}')
    for t in titles:
        adult = t.find('span', {'class': "ico_comm ico_adult"})
        
        if adult is not None: # 성인물일 경우
            idx = t.text[2:]
            title.append(idx)
            age[idx] = 19
            #print(f'19 {idx}')
       
        else: # 연령제한이 없는 경우
            idx = t.text    
            title.append(idx)
            age[idx] = 0
            #print(t.text)
        
    # author
    authors = bsObject.find_all('span', {'class':'txt_info'})
    for a in enumerate(authors):
        idx, name = a
        author[title[total_tc+idx]] = name.text[2:]
        #print(name.text[2:])
    
    # thumbnail
    thumbnails = bsObject.find_all('img', {'class': 'img_thumb'})
    for i in range(5,now_tc+5): # 0-4: 배너에 떠있는 웹툰
        idx = i-5
        thumb = thumbnails[i]['src']
        thumbnail[title[total_tc + idx]] = thumb
        #print(thumb)
    
    
    # 세부 주소 저장
    newlink_match = bsObject.find_all('a', {'class': 'link_wt'})
    for idx in range(0, now_tc):
        newlink = url[:-1] + newlink_match[idx]['href']
        website[title[total_tc+idx]] = newlink
        print(newlink)
        
    # 세부 주소으로 이동 (장르, 줄거리, 평점)
    for pIdx in range(0, now_tc):
        login_check = False

        page = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))[:-10]
        print(d, pIdx, len(page))
        driver.execute_script("arguments[0].click();", page[pIdx])
        
        link = driver.current_url

        # 성인물인 경우 로그인이 필요함
        if link[8:14] == "logins":
            login_check = True
            driver.find_element_by_css_selector('#mArticle > div.section_login > div > div > div.login_account > a.link_login.link_klogin').click()
            driver.find_element_by_name('email').send_keys(email)
            driver.find_element_by_name('password').send_keys(pw)
            driver.find_element_by_css_selector('#login-form > fieldset > div.wrap_btn > button.btn_g.btn_confirm.submit').click()

        # 다 뜰때까지 존버
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_genre')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_story')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'emph_grade')))
        
        html =  driver.page_source
        bsObject = BeautifulSoup(html, 'html.parser')

        # 장르
        genres = bsObject.find('dd', {'class': 'txt_genre'}).text.strip()
        genre[title[total_tc + pIdx]] = genres
        #print(genre)

        # 줄거리
        stories = bsObject.find('dd', {'class' : 'txt_story'}).text
        story[title[total_tc + pIdx]] = stories
        #print(story)

        # 평점
        rates = bsObject.find('em', {'class':'emph_grade'})
        rate[title[total_tc + pIdx]] = rates.text
        #print(rates.text)

        if login_check:
            driver.back()
            driver.back()
        driver.back()
        sleep(0.5)
        print(driver.current_url)
    
    total_tc += now_tc



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


now_tc=23
http://webtoon.daum.net/webtoon/view/dormitory
http://webtoon.daum.net/webtoon/view/liquidmonster
http://webtoon.daum.net/webtoon/view/dessertifyoulove
http://webtoon.daum.net/webtoon/view/dontgiveheart
http://webtoon.daum.net/webtoon/view/haksajaeseng
http://webtoon.daum.net/webtoon/view/exchangerate
http://webtoon.daum.net/webtoon/view/mujigaebridge
http://webtoon.daum.net/webtoon/view/springdays
http://webtoon.daum.net/webtoon/view/vampiredinner
http://webtoon.daum.net/webtoon/view/sayradio
http://webtoon.daum.net/webtoon/view/kangho
http://webtoon.daum.net/webtoon/view/notlover
http://webtoon.daum.net/webtoon/view/TrashStudent
http://webtoon.daum.net/webtoon/view/noticingOutsider
http://webtoon.daum.net/webtoon/view/bunnysboys
http://webtoon.daum.net/webtoon/view/blackcow
http://webtoon.daum.net/webtoon/view/drawinganoval
http://webtoon.daum.net/webtoon/view/notstopping
http://webtoon.daum.net/webtoon/view/gratia
http://webtoon.daum.net/webtoon/view/RTX
http://webtoon.dau



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


http://webtoon.daum.net/#day=mon&tab=day
now_tc=20
http://webtoon.daum.net/webtoon/view/evilknight
http://webtoon.daum.net/webtoon/view/happyhouse
http://webtoon.daum.net/webtoon/view/visionchildren
http://webtoon.daum.net/webtoon/view/gift
http://webtoon.daum.net/webtoon/view/goldhands
http://webtoon.daum.net/webtoon/view/dogcollar
http://webtoon.daum.net/webtoon/view/sunbishow
http://webtoon.daum.net/webtoon/view/loving
http://webtoon.daum.net/webtoon/view/secretrelation
http://webtoon.daum.net/webtoon/view/outsiderclub
http://webtoon.daum.net/webtoon/view/vampiresyndrome
http://webtoon.daum.net/webtoon/view/denomination
http://webtoon.daum.net/webtoon/view/parrot
http://webtoon.daum.net/webtoon/view/drfine
http://webtoon.daum.net/webtoon/view/beheaded
http://webtoon.daum.net/webtoon/view/sacreddivorce
http://webtoon.daum.net/webtoon/view/thisismyhome
http://webtoon.daum.net/webtoon/view/Matchless
http://webtoon.daum.net/webtoon/view/hellTemple
http://webtoon.daum.net/webtoon/view/di



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


...

http://webtoon.daum.net/#day=thu&tab=day
now_tc=23
http://webtoon.daum.net/webtoon/view/rejectedperson
http://webtoon.daum.net/webtoon/view/LCKsummer
http://webtoon.daum.net/webtoon/view/plusminuszero
http://webtoon.daum.net/webtoon/view/sinrakdo
http://webtoon.daum.net/webtoon/view/weather
http://webtoon.daum.net/webtoon/view/crescendo
http://webtoon.daum.net/webtoon/view/punishment
http://webtoon.daum.net/webtoon/view/previouslife
http://webtoon.daum.net/webtoon/view/breakofflove
http://webtoon.daum.net/webtoon/view/brokeup
http://webtoon.daum.net/webtoon/view/immorality
http://webtoon.daum.net/webtoon/view/callmyname
http://webtoon.daum.net/webtoon/view/unfinished
http://webtoon.daum.net/webtoon/view/yulmuartari
http://webtoon.daum.net/webtoon/view/greatchicken
http://webtoon.daum.net/webtoon/view/namnam
http://webtoon.daum.net/webtoon/view/soultoyou
http://webtoon.daum.net/webtoon/view/onlylove
http://webtoon.daum.net/webtoon/view/kimonglife
http://webtoon.daum.net/webtoon/view/pri



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


http://webtoon.daum.net/#day=fri&tab=day
now_tc=24
http://webtoon.daum.net/webtoon/view/unwelcomeguest
http://webtoon.daum.net/webtoon/view/meetcell
http://webtoon.daum.net/webtoon/view/rattrap
http://webtoon.daum.net/webtoon/view/onechoice
http://webtoon.daum.net/webtoon/view/justice
http://webtoon.daum.net/webtoon/view/beadowner
http://webtoon.daum.net/webtoon/view/weakteacher
http://webtoon.daum.net/webtoon/view/ourclass
http://webtoon.daum.net/webtoon/view/royalgame
http://webtoon.daum.net/webtoon/view/rebirth
http://webtoon.daum.net/webtoon/view/todaypropose
http://webtoon.daum.net/webtoon/view/southpole
http://webtoon.daum.net/webtoon/view/toyou2091
http://webtoon.daum.net/webtoon/view/otherworld
http://webtoon.daum.net/webtoon/view/waitU
http://webtoon.daum.net/webtoon/view/hippocrates
http://webtoon.daum.net/webtoon/view/grandmother
http://webtoon.daum.net/webtoon/view/windsword
http://webtoon.daum.net/webtoon/view/shootingstar
http://webtoon.daum.net/webtoon/view/weekendseabre



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


http://webtoon.daum.net/#day=sat&tab=day
now_tc=16
http://webtoon.daum.net/webtoon/view/symptom
http://webtoon.daum.net/webtoon/view/whitesculpture
http://webtoon.daum.net/webtoon/view/lovestory
http://webtoon.daum.net/webtoon/view/achild
http://webtoon.daum.net/webtoon/view/shamanfriend
http://webtoon.daum.net/webtoon/view/namgung
http://webtoon.daum.net/webtoon/view/bugplayer
http://webtoon.daum.net/webtoon/view/bangfamily
http://webtoon.daum.net/webtoon/view/gotowork
http://webtoon.daum.net/webtoon/view/magesdaughter
http://webtoon.daum.net/webtoon/view/surviveinharem
http://webtoon.daum.net/webtoon/view/whatasadlife
http://webtoon.daum.net/webtoon/view/favorite
http://webtoon.daum.net/webtoon/view/NormalLikeThis
http://webtoon.daum.net/webtoon/view/ficnonfic
http://webtoon.daum.net/webtoon/view/weeklyboys
sun 0 16
http://webtoon.daum.net/#day=sun&tab=day
sun 1 16
http://webtoon.daum.net/#day=sun&tab=day
sun 2 16
http://webtoon.daum.net/#day=sun&tab=day
sun 3 16
http://webtoon.daum.

In [2]:
print(len(title), len(author), len(genre), len(story), len(age), len(rate), len(thumbnail), len(website))

148 145 145 145 145 145 145 145


In [3]:
daum_webtoon = pd.DataFrame(columns = ['id', 'title', 'author', 'genre', 'platform', 'story', 'age', 'rate', 'complete', 'thumbnail', 'website'])
daum_webtoon['title'] = list(set(title)) # 중복 제거
daum_webtoon.set_index(['title'], inplace = True)
daum_webtoon.head(5)

,id,author,genre,platform,story,age,rate,complete,thumbnail,website
title,,,,,,,,,,
타원을 그리는 법,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
블랙 베히모스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
수린당 -비늘 고치는 집-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
저승사자 온라인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
아버지의 복수는 끝이 없어라,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
col = ['author', 'genre', 'story', 'age', 'rate', 'thumbnail', 'website']
dic = [author, genre, story, age, rate, thumbnail, website]
for i in range(0, len(dic)):
    for key, value in dic[i].items():
         daum_webtoon.loc[key][col[i]] = value
daum_webtoon.head(1)

,id,author,genre,platform,story,age,rate,complete,thumbnail,website
title,,,,,,,,,,
타원을 그리는 법,NaN,섬멍,"드라마, 미스터리, GL",NaN,"너무 다른 두 사람의 연애 3년, 민성의 비밀은 두 사람을 예기치 못 한 사건에 휘...",0,9.9,NaN,http://t1.daumcdn.net/webtoon/op/5604cc25c7443...,http://webtoon.daum.net/webtoon/view/drawingan...


In [5]:
daum_webtoon['complete'] = 0
daum_webtoon['platform'] = '다음 웹툰'
daum_webtoon['free'] = 1
daum_webtoon = daum_webtoon.reset_index().rename(columns={"index": ""})
daum_webtoon = daum_webtoon[['id', 'title', 'author', 'genre', 'platform', 'story', 'age', 'rate', 'free', 'complete', 'thumbnail', 'website']]

In [6]:
daum_webtoon.head(1)

,id,title,author,genre,platform,story,age,rate,free,complete,thumbnail,website
0,NaN,타원을 그리는 법,섬멍,"드라마, 미스터리, GL",다음 웹툰,"너무 다른 두 사람의 연애 3년, 민성의 비밀은 두 사람을 예기치 못 한 사건에 휘...",0,9.9,1,0,http://t1.daumcdn.net/webtoon/op/5604cc25c7443...,http://webtoon.daum.net/webtoon/view/drawingan...


In [8]:
daum_webtoon.to_csv("daum_webtoon.csv", mode='w', encoding = 'euc-kr')